In [1]:
%matplotlib notebook
"""Script to plot contour lines of bbob-constrained test functions in 2D
"""
import cocoex
import numpy as np
import matplotlib.pyplot as pp
from statsmodels.distributions.empirical_distribution import ECDF
from cocopp import ppfig

In [2]:
function = 1
dimension = 2
instance = 1
xmin, ymin = -5e3, -5
xmax, ymax = 5e3, 5

nb_cons = ["1", "2", "6", "6+n/2", "6+n", "6+3n"]
nb_cons_fig_name = ["1", "2", "6", "6_plus_n_half", "6_plus_n", "6_plus_3_n"]
obj_fcts = ["Sphere", "Sep. Ellipsoid", "Linear Slope", "Rotated Ellipsoid",\
             "Discus", "Bent Cigar", "Different Powers", "Separable Rastrigin"]

cocoex.known_suite_names.append('bbob-constrained')
cocoex.known_suite_names.append('bbob-constrained-active-only')

In [3]:
def bbob_constrained_evaluate_grid(x, y, problem):
    """Evaluate objective and constraint functions of problem
    on the grid whose x-axis is x and y-axis is y. If problem
    dimension is larger than 2, x and y are interpreted as
    first two variables, the others are set to optimum.
    
    Result:
    f_values: array of shape (nx, ny)
    cons_values: array of shape (ny, nx, problem.number_of_constraints)
    """
    f_values = []
    cons_values = []
    if problem.dimension is 3:
        Ex = ValueError()
        Ex.strerror = "Dimension cannot be 3 for the moment."
        raise Ex
    problem._best_parameter('print')
    x_opt = np.loadtxt('._bbob_problem_best_parameter.txt')
        
    for yy in y:
        # For each element xx, yy of x, y respectively,
        # evaluate objective and constraints on each point (xx, yy)
        
        # The resulting array must be of size (ny, xy)
        # for ``contour()`` to run correctly
        
        # Evaluating constraints on one point results in an array
        # of size ``problem.number_of_constraints``
        
        # todo: make sure that it also works in dimension 3
        f_values.append([problem([xx, yy] + list(x_opt[2:])) for xx in x])
        cons_values.append([problem.constraint([xx, yy] + list(x_opt[2:])) for xx in x])
    return np.array(f_values), np.array(cons_values)

In [4]:
def read_best_parameter(problem):
    """This function reads the best parameter of ``problem''
    from a .txt file and returns a array corresponding to
    this best parameter
    """
    problem._best_parameter(what="print")
    with open("._bbob_problem_best_parameter.txt") as f:
        best_parameter = f.read().strip().split()
    return np.array([float(x) for x in best_parameter])

In [5]:
def bbob_constrained_plot_contours(x, y, f_values, cons_values, x_best, function, dimension, instance):
    """Plot contour lines and best parameter of bbob-constrained
    function ``function``, in ``dimension``D, for instance ``instance``
    """
    # Set color maps for objective
    pp.rcParams['image.cmap'] = 'autumn'
    
    # Plot contours of objective
    pp.contour(x, y, f_values, 50, linewidths=0.5)
    
    # Set color maps for constraints
    pp.rcParams['image.cmap'] = 'winter'
    
    # Plot contours of each constraint
    # To indicate feasible space, positive level sets are filled with color
    for i in range(cons_values.shape[-1]):
        pp.contourf(x, y, np.array(cons_values)[:, :, i], levels=[0, 1e12], alpha=0.2)
        
    # Plot best parameter
    pp.plot(x_best[0], x_best[1], 'o', c='black', markersize=10)
    
    # Annotate best parameter on the graph
    if x_best[0] < -3.3:
        xshift = -0.3
    else:
        xshift = 1.7
    pp.annotate(r'$x^{opt}$', xy=(x_best[0], x_best[-1]), xytext=((x_best[0]-xshift), x_best[1]+0.1), size=25)
    # Prepare figure settings
    f = (function - 1) // 6
    m = (function - 1) % 6
    cons = "constraints"
    if m == 0:
        cons = "constraint"
    title = "%d %s with %s act. %s, inst.=%s"%(function, obj_fcts[f], nb_cons[m], cons, instance)
    pp.title(title)
    ax = pp.gca()
    ax.set_aspect('equal') # commented for testing purpose
    #pp.xlabel(r'$x_1$', size=15)
    #pp.ylabel(r'$x_2$', size=15)
    pp.grid(which ='both', linestyle='dotted')
    pp.tick_params(axis='both', labelsize=12)
    #pp.legend()
    #pp.show()
    ppfig.save_figure("contours_f%02d_%02d_i%02d-%s_constraints" % (function, 
                                                                    dimension, 
                                                                    instance, 
                                                                    nb_cons_fig_name[m]),
                      bbox_inches='tight')
    pp.clf()

In [6]:
def main(function=1, dimension=2, instance=1):
    # Create bbob-constrained test suite
    suite = cocoex.Suite("bbob-constrained", "", "")
    #suite = cocoex.Suite("bbob-constrained-active-only", "", "")
    
    # Create the constrained problem of interest
    problem = suite.get_problem_by_function_dimension_instance(function, dimension,
                                                           instance)
    # Read and evaluate best parameter of the problem
    x_best = read_best_parameter(problem)
    f_best = problem(x_best)
    
    # Define x- and y-axes
    x = np.linspace(xmin, xmax, 100)
    y = np.linspace(ymin, ymax, 100)
    
    # Evaluate objective on grid
    f_values, cons_values = bbob_constrained_evaluate_grid(x, y, problem)
    
    # Plot contours of objective and constraints
    bbob_constrained_plot_contours(x, y, f_values, cons_values, x_best, function, dimension, instance)

In [7]:
if __name__=='__main__':
    main(function, dimension, instance)
    pp.show()

In [ ]:
# contour plots for f1-48 in dimension 2:

functions = range(1, 49)
dimensions = [20, ]
instances = range(1, 2)

xmin, ymin = -5, -5
xmax, ymax = 5, 5

for function in functions:
    for dimension in dimensions:
        for instance in instances:
            main(function, dimension, instance)
